In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Check for GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Set image dimensions and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Load dataset using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the data from your paths
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/sdss_mobile/dataset_split/Train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/sdss_mobile/dataset_split/Validation',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/sdss_mobile/dataset_split/Test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Calculate class weights for imbalanced datasets
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Load DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout to reduce overfitting
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes for classification

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Save model checkpoint callback
checkpoint = ModelCheckpoint(
    'best_model_densenet.keras',  # Filepath where the model will be saved
    monitor='val_loss',  # Monitor validation loss
    save_best_only=True,  # Save only the best model
    mode='min',  # Save model when validation loss decreases
    verbose=1
)

# Reduce learning rate callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=20,
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stop]  # Add the callbacks here
)

# Unfreeze more layers for fine-tuning (start from layer 200)
for layer in base_model.layers[:200]:
    layer.trainable = False
for layer in base_model.layers[200:]:
    layer.trainable = True

# Recompile model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tuning with more epochs
fine_tune_epochs = 30  # Increase fine-tuning epochs
total_epochs = 20 + fine_tune_epochs

# Continue fine-tuning with model saving callbacks
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    class_weight=class_weights,
    callbacks=[checkpoint, reduce_lr, early_stop]  # Add the callbacks here
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print('Test accuracy:', test_acc)

# Classification report and confusion matrix
Y_pred = model.predict(test_generator, test_generator.samples // BATCH_SIZE + 1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = ['Class 5 - Barred Spiral Galaxies', 'Class 6 - Unbarred Tight Spiral Galaxies', 'Class 7 - Unbarred Loose Spiral Galaxies']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Found GPU at: /device:GPU:0
Found 2479 images belonging to 3 classes.
Found 460 images belonging to 3 classes.
Found 650 images belonging to 3 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5034 - loss: 0.9743 
Epoch 1: val_loss improved from inf to 3.41294, saving model to best_model_densenet.keras
77/77 ━━━━━━━━━━━━━━━━━━━━ 903s 11s/step - accuracy: 0.5037 - loss: 0.9734 - val_accuracy: 0.2857 - val_loss: 3.4129 - learning_rate: 1.0000e-04
Epoch 2/20
 1/77 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4375 - loss: 0.8263

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 3.41294
77/77 ━━━━━━━━━━━━━━━━━━━━ 13s 175ms/step - accuracy: 0.4375 - loss: 0.8263 - val_accuracy: 0.2500 - val_loss: 3.4496 - learning_rate: 1.0000e-04
Epoch 3/20
76/77 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - accuracy: 0.5694 - loss: 0.8147
Epoch 3: val_loss did not improve from 3.41294
77/77 ━━━━━━━━━━━━━━━━━━━━ 32s 385ms/step - accuracy: 0.5694 - loss: 0.8142 - val_accuracy: 0.3147 - val_loss: 3.5824 - learning_rate: 1.0000e-04
Epoch 4/20
 1/77 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5000 - loss: 1.0488
Epoch 4: val_loss did not improve from 3.41294
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.5000 - loss: 1.0488 - val_accuracy: 0.2500 - val_loss: 3.7639 - learning_rate: 1.0000e-04
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.5676 - loss: 0.7752
Epoch 5: val_loss did not improve from 3.41294
77/77 ━━━━━━━━━━━━━━━━━━━━ 32s 380ms/step - accuracy: 0.5680 - loss: 0.7749 - val_accuracy: 0.3393 - val_loss: 3.7882

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
